# Objetivo

Puxar os dados diários e converter em mensais. Manter apenas papéis com setores associados. Renomear colunas.

In [13]:
# Bibliotecas
import glob
import numpy as np
import pandas as pd
from scipy.stats import iqr

# Funções

In [2]:
def encontrar_setor(papel, df_setor):
    '''
    Função para encontrar o setor de um papel.
    '''
    for setor, papeis_setor in zip(df_setor['SETOR_NOME'], df_setor['PAPEIS']):
        if papel in papeis_setor:
            return setor
    return 'NDA'

In [3]:
def tratar_extracao_diaria(ano, mes, df_ext, df_setor):
    '''
    Função para unificar extrações diarias em um só dataframe.
    '''
    # Listar papéis que possuem algum setor
    filtro_papeis = list(set([papel for papeis in df_setor['PAPEIS'] for papel in papeis]))
    len(filtro_papeis)
    
    # Tratar colunas
    df_ext.columns = [col.replace(' ','').upper() for col in df_ext.columns]
    
    # Checar - Quantos papéis com setor existem na extracao
    #df_ext['PAPEL'].isin(filtro_papeis).sum()
    
    # Filtrar papeis que possuem setores
    df_tra = df_ext[df_ext['PAPEL'].str.replace(' ','').isin(filtro_papeis)].copy()

    # Definir setor
    df_tra['SETOR'] = df_tra['PAPEL'].apply(lambda p: encontrar_setor(p, df_setor))

    # Adicionar colunas de mês e ano
    df_tra['ANO'] = ano
    df_tra['MES'] = mes
    
    # Reordenar
    cols_def = ['PAPEL', 'SETOR', 'ANO', 'MES']
    cols = cols_def + [c for c in df_tra.columns if c not in cols_def]
    df_tra = df_tra[cols].copy()
    
    # Retorno
    return df_tra

# Execução

In [5]:
# Definir parâmetros
ANO = '2023'
MES = '09'

In [6]:
# Puxar extracoes diarias a partir de parâmetros
lista_ext_dia = list(sorted(glob.glob(f'./dados/1_EXT/fundamentalista_br_{ANO}-{MES}*')))
setor_ano_mes = list(sorted(glob.glob(f'./dados/1_EXT/setores_papeis_{ANO}-{MES}*')))[-1]

In [7]:
# Puxar e atuar em base de setores
df_setor = pd.read_parquet(setor_ano_mes)
df_setor = df_setor[df_setor['SETOR_NOME'] != '???'].reset_index(drop=True).copy()
_ = df_setor['PAPEIS'].str.split('###').tolist()
df_setor['PAPEIS'] = _

# Listagem
l_papeis_tra = list()

# Loop
for papel_diario in lista_ext_dia:
    # Puxar dia mês ano
    ano, mes, dia = papel_diario.split('.')[-2].split('_')[-1].split('-')
    
    # Puxar bases de dados - Melhorar com "try except"
    df_ext = pd.read_parquet(papel_diario)
    
    # Tratar
    l_papeis_tra.append(tratar_extracao_diaria(ano, mes, df_ext, df_setor))

In [8]:
# Unificar base diarias tratadas
df_papeis_dia = pd.concat(l_papeis_tra).sort_values('PAPEL')

In [14]:
# Gerar base mensal com estatisticas descritvas
df_papeis_mes_media = df_papeis_dia.groupby(['PAPEL', 'SETOR', 'ANO', 'MES']).agg(np.mean).reset_index()
df_papeis_mes_mediana = df_papeis_dia.groupby(['PAPEL', 'SETOR', 'ANO', 'MES']).agg(np.median).reset_index()
df_papeis_mes_std = df_papeis_dia.groupby(['PAPEL', 'SETOR', 'ANO', 'MES']).agg(np.std).reset_index()
df_papeis_mes_iqr = df_papeis_dia.groupby(['PAPEL', 'SETOR', 'ANO', 'MES']).agg(iqr).reset_index()

In [17]:
# Salvar transformações
df_papeis_mes_media.to_parquet(f'./dados/2_TRA/papeis_{ANO}_{MES}_media.parquet')
df_papeis_mes_mediana.to_parquet(f'./dados/2_TRA/papeis_{ANO}_{MES}_mediana.parquet')
df_papeis_mes_std.to_parquet(f'./dados/2_TRA/papeis_{ANO}_{MES}_desvio.parquet')
df_papeis_mes_iqr.to_parquet(f'./dados/2_TRA/papeis_{ANO}_{MES}_iqr.parquet')